In [ ]:
# Spark Session
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder.appName("Basic Transformation - I")
    .master("spark://spark-master:7077")
    .getOrCreate()
)

spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/17 16:00:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [19]:
spark.sparkContext.defaultParallelism

8

In [ ]:
# Emp Data & Schema

emp_data = [
    ["001", "101", "John Doe", "30", "Male", "50000", "2015-01-01"],
    ["002", "101", "Jane Smith", "25", "Female", "45000", "2016-02-15"],
    ["003", "102", "Bob Brown", "35", "Male", "55000", "2014-05-01"],
    ["004", "102", "Alice Lee", "28", "Female", "48000", "2017-09-30"],
    ["005", "103", "Jack Chan", "40", "Male", "60000", "2013-04-01"],
    ["006", "103", "Jill Wong", "32", "Female", "52000", "2018-07-01"],
    ["007", "101", "James Johnson", "42", "Male", "70000", "2012-03-15"],
    ["008", "102", "Kate Kim", "29", "Female", "51000", "2019-10-01"],
    ["009", "103", "Tom Tan", "33", "Male", "58000", "2016-06-01"],
    ["010", "104", "Lisa Lee", "27", "Female", "47000", "2018-08-01"],
    ["011", "104", "David Park", "38", "Male", "65000", "2015-11-01"],
    ["012", "105", "Susan Chen", "31", "Female", "54000", "2017-02-15"],
    ["013", "106", "Brian Kim", "45", "Male", "75000", "2011-07-01"],
    ["014", "107", "Emily Lee", "26", "Female", "46000", "2019-01-01"],
    ["015", "106", "Michael Lee", "37", "Male", "63000", "2014-09-30"],
    ["016", "107", "Kelly Zhang", "30", "Female", "49000", "2018-04-01"],
    ["017", "105", "George Wang", "34", "Male", "57000", "2016-03-15"],
    ["018", "104", "Nancy Liu", "29", "Female", "50000", "2017-06-01"],
    ["019", "103", "Steven Chen", "36", "Male", "62000", "2015-08-01"],
    ["020", "102", "Grace Kim", "32", "Female", "53000", "2018-11-01"],
]

emp_schema = "employee_id string, department_id string, name string, age string, gender string, salary string, hire_date string"

In [3]:
# Create emp DataFrame

emp = spark.createDataFrame(data=emp_data, schema=emp_schema)

In [4]:
# Show emp dataframe (ACTION)

emp.show()

+-----------+-------------+-------------+---+------+------+----------+
|employee_id|department_id|         name|age|gender|salary| hire_date|
+-----------+-------------+-------------+---+------+------+----------+
|        001|          101|     John Doe| 30|  Male| 50000|2015-01-01|
|        002|          101|   Jane Smith| 25|Female| 45000|2016-02-15|
|        003|          102|    Bob Brown| 35|  Male| 55000|2014-05-01|
|        004|          102|    Alice Lee| 28|Female| 48000|2017-09-30|
|        005|          103|    Jack Chan| 40|  Male| 60000|2013-04-01|
|        006|          103|    Jill Wong| 32|Female| 52000|2018-07-01|
|        007|          101|James Johnson| 42|  Male| 70000|2012-03-15|
|        008|          102|     Kate Kim| 29|Female| 51000|2019-10-01|
|        009|          103|      Tom Tan| 33|  Male| 58000|2016-06-01|
|        010|          104|     Lisa Lee| 27|Female| 47000|2018-08-01|
|        011|          104|   David Park| 38|  Male| 65000|2015-11-01|
|     

In [5]:
# Schema for emp

emp.schema

StructType([StructField('employee_id', StringType(), True), StructField('department_id', StringType(), True), StructField('name', StringType(), True), StructField('age', StringType(), True), StructField('gender', StringType(), True), StructField('salary', StringType(), True), StructField('hire_date', StringType(), True)])

In [ ]:
# Small Example for Schema
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

schema_string = "name string, age int"

schema_spark = StructType(
    [StructField("name", StringType(), True), StructField("age", IntegerType(), True)]
)

In [9]:
# Columns and expression
import pyspark.sql.functions as F

print(emp.salary)
print(emp["salary"])
print(F.col("salary"))
print(F.expr("salary"))

Column<'salary'>
Column<'salary'>
Column<'salary'>
Column<'salary'>


In [10]:
# SELECT columns
# select employee_id, name, age, salary from emp

emp_filtered = emp.select(F.col("employee_id"), F.expr("name"), emp.age, emp["salary"])

In [8]:
# SHOW Dataframe (ACTION)

emp_filtered.show()

+-----------+-------------+---+------+
|employee_id|         name|age|salary|
+-----------+-------------+---+------+
|        001|     John Doe| 30| 50000|
|        002|   Jane Smith| 25| 45000|
|        003|    Bob Brown| 35| 55000|
|        004|    Alice Lee| 28| 48000|
|        005|    Jack Chan| 40| 60000|
|        006|    Jill Wong| 32| 52000|
|        007|James Johnson| 42| 70000|
|        008|     Kate Kim| 29| 51000|
|        009|      Tom Tan| 33| 58000|
|        010|     Lisa Lee| 27| 47000|
|        011|   David Park| 38| 65000|
|        012|   Susan Chen| 31| 54000|
|        013|    Brian Kim| 45| 75000|
|        014|    Emily Lee| 26| 46000|
|        015|  Michael Lee| 37| 63000|
|        016|  Kelly Zhang| 30| 49000|
|        017|  George Wang| 34| 57000|
|        018|    Nancy Liu| 29| 50000|
|        019|  Steven Chen| 36| 62000|
|        020|    Grace Kim| 32| 53000|
+-----------+-------------+---+------+



In [ ]:
# Using expr for select
# select employee_id as emp_id, name, cast(age as int) as age, salary from emp_filtered

emp_casted = emp_filtered.select(
    F.expr("employee_id as emp_id"),
    emp.name.alias("alias_name"),
    F.expr("cast(age as int) as age"),
    emp["salary"].cast("double").alias("double_salary"),
)

In [12]:
# SHOW Dataframe (ACTION)

emp_casted.show()

+------+-------------+---+-------------+
|emp_id|   alias_name|age|double_salary|
+------+-------------+---+-------------+
|   001|     John Doe| 30|      50000.0|
|   002|   Jane Smith| 25|      45000.0|
|   003|    Bob Brown| 35|      55000.0|
|   004|    Alice Lee| 28|      48000.0|
|   005|    Jack Chan| 40|      60000.0|
|   006|    Jill Wong| 32|      52000.0|
|   007|James Johnson| 42|      70000.0|
|   008|     Kate Kim| 29|      51000.0|
|   009|      Tom Tan| 33|      58000.0|
|   010|     Lisa Lee| 27|      47000.0|
|   011|   David Park| 38|      65000.0|
|   012|   Susan Chen| 31|      54000.0|
|   013|    Brian Kim| 45|      75000.0|
|   014|    Emily Lee| 26|      46000.0|
|   015|  Michael Lee| 37|      63000.0|
|   016|  Kelly Zhang| 30|      49000.0|
|   017|  George Wang| 34|      57000.0|
|   018|    Nancy Liu| 29|      50000.0|
|   019|  Steven Chen| 36|      62000.0|
|   020|    Grace Kim| 32|      53000.0|
+------+-------------+---+-------------+



In [ ]:
emp_casted_1 = emp_filtered.selectExpr(
    "employee_id as emp_id", "name", "cast(age as int) as age", "salary"
)

In [14]:
emp_casted_1.show()

+------+-------------+---+------+
|emp_id|         name|age|salary|
+------+-------------+---+------+
|   001|     John Doe| 30| 50000|
|   002|   Jane Smith| 25| 45000|
|   003|    Bob Brown| 35| 55000|
|   004|    Alice Lee| 28| 48000|
|   005|    Jack Chan| 40| 60000|
|   006|    Jill Wong| 32| 52000|
|   007|James Johnson| 42| 70000|
|   008|     Kate Kim| 29| 51000|
|   009|      Tom Tan| 33| 58000|
|   010|     Lisa Lee| 27| 47000|
|   011|   David Park| 38| 65000|
|   012|   Susan Chen| 31| 54000|
|   013|    Brian Kim| 45| 75000|
|   014|    Emily Lee| 26| 46000|
|   015|  Michael Lee| 37| 63000|
|   016|  Kelly Zhang| 30| 49000|
|   017|  George Wang| 34| 57000|
|   018|    Nancy Liu| 29| 50000|
|   019|  Steven Chen| 36| 62000|
|   020|    Grace Kim| 32| 53000|
+------+-------------+---+------+



In [14]:
emp_casted.printSchema()

root
 |-- emp_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- salary: string (nullable = true)



In [16]:
# Filter emp based on Age > 30
# select emp_id, name, age, salary from emp_casted where age > 30

emp_final = emp_casted_1.select("emp_id", "name", "age", "salary").where("age > 30")

In [17]:
# SHOW Dataframe (ACTION)

emp_final.show()

+------+-------------+---+------+
|emp_id|         name|age|salary|
+------+-------------+---+------+
|   003|    Bob Brown| 35| 55000|
|   005|    Jack Chan| 40| 60000|
|   006|    Jill Wong| 32| 52000|
|   007|James Johnson| 42| 70000|
|   009|      Tom Tan| 33| 58000|
|   011|   David Park| 38| 65000|
|   012|   Susan Chen| 31| 54000|
|   013|    Brian Kim| 45| 75000|
|   015|  Michael Lee| 37| 63000|
|   017|  George Wang| 34| 57000|
|   019|  Steven Chen| 36| 62000|
|   020|    Grace Kim| 32| 53000|
+------+-------------+---+------+



In [18]:
# Write the data back as CSV (ACTION)

emp_final.write.format("parquet").save("hdfs://namenode:9000/output/basics/05/emp")

In [ ]:
# benchmark de escritura

emp_final.write.mode("overwrite").format("noop").save()

In [23]:
spark.stop()